In [73]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm

import random
import numpy as np
import pandas as pd

import pickle
import datetime as dt
import time

import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import cv2

from keras.callbacks import TensorBoard

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from mpl_toolkits.axes_grid1 import ImageGrid

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.models import Sequential
from keras.models import Model


from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau


from keras import backend as K

from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3

from keras.applications.inception_v3 import InceptionV3

from keras.applications.vgg16 import preprocess_input, decode_predictions

from keras.optimizers import SGD




np.random.seed(2)
sns.set(style='white', context='notebook', palette='deep')

In [74]:
def show_pred(preds, Y, val_breed, index, seq, ran):
    leng = len(preds)
    if seq:
        for i in range(index):
            if ran:
                index = random.randint(0, leng) 
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
    else:
            _, imagenet_class_name, prob = decode_predictions(preds, top=1)[index][0]
            plt.title("Original: " + val_breed[Y[index]] + "\nPrediction: " + imagenet_class_name)
            plt.imshow(X_train[index])
            plt.show()
        
def accuracy_func(preds, Y, val_breed):
    leng = len(preds)
    count = 0;
    for i in range(leng):
        _, imagenet_class_name, prob = decode_predictions(preds, top=1)[i][0]
        if val_breed[Y[i]] == imagenet_class_name:
            count+=1
    accuracy = (count/leng)*100
    
    print("Accuracy: ", accuracy)
    return accuracy

In [75]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [76]:
#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet')
 
#Load the Inception_V3 model
inception_model = inception_v3.InceptionV3(weights='imagenet')
 
#Load the ResNet50 model
resnet_model = resnet50.ResNet50(weights='imagenet')
 
#Load the MobileNet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [77]:
training_path = 'data/pre/train'
validation_path = 'data/pre/validation'
testing_path = 'data/pre/test'
batch_size = 32
target_size=(224, 224)
norm = 255.0
class_mode='categorical'

In [78]:
train_datagen = ImageDataGenerator(
        rescale=1./norm,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./norm)
test_datagen = ImageDataGenerator(rescale=1./norm)

train_generator = train_datagen.flow_from_directory(
        training_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

test_generator = test_datagen.flow_from_directory(
        testing_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

Found 2657 images belonging to 120 classes.
Found 825 images belonging to 120 classes.
Found 874 images belonging to 120 classes.


In [79]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [80]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(120, activation='softmax')(x)


In [81]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [84]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True



0 input_28
1 conv2d_941
2 batch_normalization_941
3 activation_1186
4 conv2d_942
5 batch_normalization_942
6 activation_1187
7 conv2d_943
8 batch_normalization_943
9 activation_1188
10 max_pooling2d_46
11 conv2d_944
12 batch_normalization_944
13 activation_1189
14 conv2d_945
15 batch_normalization_945
16 activation_1190
17 max_pooling2d_47
18 conv2d_949
19 batch_normalization_949
20 activation_1194
21 conv2d_947
22 conv2d_950
23 batch_normalization_947
24 batch_normalization_950
25 activation_1192
26 activation_1195
27 average_pooling2d_91
28 conv2d_946
29 conv2d_948
30 conv2d_951
31 conv2d_952
32 batch_normalization_946
33 batch_normalization_948
34 batch_normalization_951
35 batch_normalization_952
36 activation_1191
37 activation_1193
38 activation_1196
39 activation_1197
40 mixed0
41 conv2d_956
42 batch_normalization_956
43 activation_1201
44 conv2d_954
45 conv2d_957
46 batch_normalization_954
47 batch_normalization_957
48 activation_1199
49 activation_1202
50 average_pooling2d_92


In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])

Error in sys.excepthook:
Traceback (most recent call last):
  File "C:\Users\Andromeda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1776, in excepthook
    self.showtraceback((etype, value, tb), tb_offset=0)
KeyboardInterrupt

Original exception was:
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
    PyErr_CheckSignals()
KeyboardInterrupt
Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
    PyErr_CheckSignals()
KeyboardInterrupt
Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x0000017ABE014A80> >>
Traceback (most recent call last):
  File "C:\Users\Andromeda\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.p

In [ ]:
tensorboard = TensorBoard(log_dir='output/logs', histogram_freq=1, write_graph=True, write_images=False)
tensorboard.set_model(model) 

In [ ]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit_generator(
    train_generator,
    steps_per_epoch=83,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=25,
    verbose=1,
    callbacks=[tensorboard])

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [22]:
preds = model.predict_generator(test_generator, verbose=1)

28/28 [==============================] - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 58s - ETA: 46 - ETA: 34 - ETA: 23 - ETA: 11 - 316s 11s/step


In [23]:
preds

array([[0.00385027, 0.00223356, 0.00263116, ..., 0.00190942, 0.02419008,
        0.0052945 ],
       [0.00362196, 0.00516471, 0.00250237, ..., 0.00208397, 0.01137248,
        0.00702833],
       [0.00315248, 0.00420116, 0.00483776, ..., 0.00340643, 0.01122256,
        0.00459765],
       ...,
       [0.00493188, 0.00444942, 0.00286971, ..., 0.00207865, 0.01424604,
        0.00548674],
       [0.00493173, 0.00577004, 0.00434456, ..., 0.00548583, 0.00592475,
        0.00651255],
       [0.0099298 , 0.00679917, 0.00645528, ..., 0.00247888, 0.00785184,
        0.00897504]], dtype=float32)